In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from glob import glob

In [2]:
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Model

Using TensorFlow backend.


In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
num_encoder_tokens = len(word2ind)
num_decoder_tokens = len(word2ind)
emb_dim = 100
latent_dim = 100

In [6]:
num_encoder_tokens

76

In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, emb_dim)
encoder_embedding_outputs = BatchNormalization()(encoder_embedding(encoder_inputs))
encoder = LSTM(latent_dim, return_state=True, dropout=0.2, recurrent_dropout=0.2)
encoder_outputs, state_h, state_c = encoder(encoder_embedding_outputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_embedding = Embedding(num_decoder_tokens, emb_dim)
decoder_embedding_outputs = BatchNormalization()(decoder_embedding(decoder_inputs))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,
                    dropout=0.2, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_outputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [8]:
model.load_weights('model_weights.h5')

In [9]:
# Run training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
train_data_start = 36
train_data_end = 37
encoder_input_files = sorted(glob('data/encoder_*'))[train_data_start:train_data_end]
decoder_input_files = sorted(glob('data/decoder_input_*'))[train_data_start:train_data_end]
decoder_target_files = sorted(glob('data/decoder_target_*'))[train_data_start:train_data_end]

In [11]:
encoder_input_data = pd.DataFrame(None)
decoder_input_data = pd.DataFrame(None)
decoder_target_data= pd.DataFrame(None)
for encoder_input_file, decoder_input_file, decoder_target_file in zip(encoder_input_files, decoder_input_files, decoder_target_files):
    print(encoder_input_file)
    encoder_input_data = encoder_input_data.append(pd.read_csv(encoder_input_file))
    decoder_input_data = decoder_input_data.append(pd.read_csv(decoder_input_file))
    decoder_target_data = decoder_target_data.append(pd.read_csv(decoder_target_file))

data/encoder_input_36.csv


In [12]:
encoder_input_data = encoder_input_data.iloc[:, list(range(encoder_input_data.shape[1]-1,-1,-1))].values
decoder_input_data = decoder_input_data.values
decoder_target_data_2d = decoder_target_data.values

In [13]:
decoder_target_data = decoder_target_data_2d.reshape((*decoder_target_data_2d.shape, -1))

In [14]:
def train_valid_split(length, test_size=0.3, random_state=43):
    np.random.RandomState(seed=random_state)
    choices = list(range(length))
    val_choices = np.random.choice(
        choices,
        int(length*test_size),
        replace=False
    ).tolist()
    train_choices = list(set(choices) - set(val_choices))
    return train_choices, val_choices

In [15]:
encoder_input_data.shape

(1121707, 100)

In [16]:
decoder_input_data.shape

(1121707, 100)

In [17]:
decoder_target_data.shape

(1121707, 100, 1)

In [18]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, None, 100)     7600        input_1[0][0]                    
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, None, 100)     7600        input_2[0][0]                    
___________________________________________________________________________________________

In [19]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=500,
          epochs=1,
          validation_split=0.1,
          shuffle=True)

Train on 1009536 samples, validate on 112171 samples
Epoch 1/1
1009536/1009536 [==============================] - 351s - loss: 1.4023 - acc: 0.5780 - val_loss: 1.5357 - val_acc: 0.5476


In [20]:
model.save_weights('model_weights.h5')

In [21]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, emb_dim)
encoder_embedding_outputs = BatchNormalization()(encoder_embedding(encoder_inputs))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding_outputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_embedding = Embedding(num_decoder_tokens, emb_dim)
decoder_embedding_outputs = BatchNormalization()(decoder_embedding(decoder_inputs))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_outputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.load_weights('model_weights.h5')

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_outputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.array(word2ind[' ']).reshape((1, 1))

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        first_n = np.argsort(output_tokens[0, -1, :])[-3:]
        probs = output_tokens[0, -1, first_n] / np.sum(output_tokens[0, -1, first_n])
        sampled_token_index = np.random.choice(first_n, p=probs)
        # sampled_token_index = np.random.choice(list(range(num_decoder_tokens)), p=output_tokens[0, -1, :])
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_sentence) > 200:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.array(sampled_token_index).reshape((1, 1))

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [24]:
print(decode_sequence(encoder_input_data[0]))

like a care and shoes  
  
And I say I've got to be that a cales  
I seem to you the such and the back of the same  
  
And it's not to be together  
I'm not all the sun  
I can't be the stranger and t
